# Compare hologram data quality on PWV

- author Sylvie Dagoret-Campagne
- creation date 2024-09-25 
- last update : 2024-09-25 

- affiliation : IJCLab
- Kernel @usdf **w_2024_37**
- Office emac : mamba_py311
- Home emac : base (conda)
- laptop : conda_py310


In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figsPWV"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit,least_squares


# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove to run faster the notebook
import ipywidgets as widgets
%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

### Spectroscopy

In [ ]:
version_results = "v3"

In [ ]:
if version_results == "":
    atmfilename = "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_testWithMaskedEdges_newBoundaries_newPolysRescaled_newFitBounds_adjustA1_lockedOrder2_removeThroughputTails_2.npy"
elif version_results == "v2":
    atmfilename = "auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_PeekFinder.npy"
elif version_results == "v3":
    atmfilename = "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z.npy"

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)
df_spec

In [ ]:
#subtract the a bug number to have normal dates
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)
df_spec["nightObs"]

In [ ]:
ser_spec_size = df_spec.groupby(["nightObs"]).size()

In [ ]:
ser_spec_size

In [ ]:
type(ser_spec_size)

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(ser_spec_size.values,bins=50,facecolor="b")
ax.set_title("nb obs per night")
ax.set_xlabel("Nobs/night")

In [ ]:
ser_CHI2_FIT = df_spec[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV = df_spec[["PWV [mm]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV_CHI2_FIT = df_spec[["PWV [mm]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
ser_PWV

In [ ]:
ser_CHI2_FIT

In [ ]:
ser_PWV_CHI2_FIT 

### Plot PWV and Chi2 before anly selection

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title="Number of measurements per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Mean PWV per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Median PWV per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="STD variation for PWV per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="r",grid=True,title="Number of measurements per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Mean CHI2 per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Median CHI2 per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="STD variation CHI2 per night")
plt.tight_layout()

In [ ]:
ser_PWV_CHI2_FIT 

In [ ]:
ser_PWV_CHI2_FIT.loc[20230117,:]

In [ ]:
df_unstack = ser_PWV_CHI2_FIT.loc[20230117,:].unstack()
df_unstack

In [ ]:
ser_PWV_CHI2_FIT.loc[20230117,"PWV [mm]_x"]["count"]

In [ ]:
df_unstack.loc["PWV [mm]_x","count"]

In [ ]:
def FillAgreggates(row):
    the_nightObs = row["nightObs"]
    df_night = ser_PWV_CHI2_FIT.loc[the_nightObs,:].unstack()
    count = df_night.loc["PWV [mm]_x","count"]
    pwvmin = df_night.loc["PWV [mm]_x","min"]
    pwvmax = df_night.loc["PWV [mm]_x","max"]
    pwvmean = df_night.loc["PWV [mm]_x","mean"]
    pwvmedian = df_night.loc["PWV [mm]_x","median"]
    pwvstd = df_night.loc["PWV [mm]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_pwvmin":pwvmin,"_pwvmax":pwvmax,"_pwvmean":pwvmean,"_pwvmedian":pwvmedian,"_pwvstd":pwvstd,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    
    ser = pd.Series(data=d)

    return ser
    

In [ ]:
row = df_spec.iloc[0]

In [ ]:
row

In [ ]:
FillAgreggates(row)

In [ ]:
df_spec = df_spec.join(df_spec.apply(FillAgreggates,axis=1,result_type="expand"))

In [ ]:
df_spec

## Aply selection

In [ ]:
fig,ax = plt.subplots(1,1)
df_spec["CHI2_FIT"].hist(bins=50,ax=ax,range=(0,200))
ax.set_yscale("log")

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
#df_spec["PWV [mm]_x"]

In [ ]:
Dt = pd.Timedelta(minutes=30)
d = {}
list_of_nightobs = df_spec["nightObs"].unique()
for nightobs in list_of_nightobs:
    sel_flag = df_spec["nightObs"] == nightobs
    df_night = df_spec[sel_flag]
    tmin = df_night["Time"].min()-Dt
    tmax = df_night["Time"].max()+Dt
    d[nightobs] = (tmin,tmax)

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,4))
ax1  = axs
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="PWV_x")
ax1.set_ylabel("PWV [mm]_x")
#df_spec.plot(x="Time",y="PWV [mm]_y",ax=ax2,marker='+',c="b",lw=0.0,grid=True,label="PWV_y")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("Precipibable water vapor meas by holo vs time")

for key, tt in d.items():
    ax1.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()

## Apply selection

In [ ]:
cut =  (df_spec["CHI2_FIT"]<20) & (df_spec["PWV [mm]_x"] >0.1) & (df_spec["PWV [mm]_x"] < 14.9) 
df_spec_sel = df_spec[cut].drop(labels=['_count', '_pwvmin', '_pwvmax', '_pwvmean', '_pwvmedian', '_pwvstd','_chi2min', '_chi2max', '_chi2mean', '_chi2median', '_chi2std'],axis=1)

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)

In [ ]:
df_spec_sel

In [ ]:
ser_PWV_CHI2_FIT_sel = df_spec_sel[["PWV [mm]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
ser_PWV_CHI2_FIT_sel

In [ ]:
def FillAgreggatesSel(row):
    the_nightObs = row["nightObs"]
    df_night = ser_PWV_CHI2_FIT_sel.loc[the_nightObs,:].unstack()
    count = df_night.loc["PWV [mm]_x","count"]
    pwvmin = df_night.loc["PWV [mm]_x","min"]
    pwvmax = df_night.loc["PWV [mm]_x","max"]
    pwvmean = df_night.loc["PWV [mm]_x","mean"]
    pwvmedian = df_night.loc["PWV [mm]_x","median"]
    pwvstd = df_night.loc["PWV [mm]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_pwvmin":pwvmin,"_pwvmax":pwvmax,"_pwvmean":pwvmean,"_pwvmedian":pwvmedian,"_pwvstd":pwvstd,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    
    ser = pd.Series(data=d)

    return ser
    

In [ ]:
row = df_spec_sel.iloc[0]
row

In [ ]:
FillAgreggatesSel(row)

In [ ]:
df_part2 = df_spec_sel.apply(FillAgreggatesSel,axis=1,result_type="expand")
df_part2

In [ ]:
df_spec_sel = df_spec_sel.join(df_spec_sel.apply(FillAgreggatesSel,axis=1,result_type="expand"))

In [ ]:
df_spec_sel

In [ ]:
Dt = pd.Timedelta(minutes=30)
d = {}
list_of_nightobs = df_spec_sel["nightObs"].unique()
for nightobs in list_of_nightobs:
    sel_flag = df_spec_sel["nightObs"] == nightobs
    df_night = df_spec_sel[sel_flag]
    tmin = df_night["Time"].min()-Dt
    tmax = df_night["Time"].max()+Dt
    d[nightobs] = (tmin,tmax)

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,4))
ax1  = axs
df_spec_sel.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="PWV_x")
ax1.set_ylabel("PWV [mm]_x")
#df_spec_sel.plot(x="Time",y="PWV [mm]_y",ax=ax2,marker='+',c="b",lw=0.0,grid=True,label="PWV_y")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("Precipibable water vapor meas by holo selected vs time")

for key, tt in d.items():
    ax1.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()

In [ ]:
ser_CHI2_FIT_sel = df_spec_sel[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV_sel = df_spec_sel[["PWV [mm]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title="Number of measurements per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Mean PWV per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Median PWV per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="STD variation for PWV per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Mean CHI2 per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Median CHI2 per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="STD variation CHI2 per night after selection")
plt.tight_layout()

## Show time variation each night

In [ ]:
all_selected_nights = df_spec_sel["nightObs"].unique()

In [ ]:
def funclineres(params, x, y, yerr):
    # Return residual = fit-observed
    return (y-params[0] -params[1]*x)/yerr
def funcline(params,x):
    return params[0] + params[1]*x

In [ ]:
all_dateObs_sel = {}

for night in all_selected_nights:
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    df_night_pwv_curve = df_spec_night[["Time","PWV [mm]_x","PWV [mm]_err_x"]]

    tmin = df_night_pwv_curve["Time"].min()

    # convert in hours
    df_night_pwv_curve["dt"] = (df_night_pwv_curve["Time"] - tmin).dt.total_seconds()/3600.
    
    
    stat = df_night_pwv_curve[["PWV [mm]_x"]].describe()
    date_form = DateFormatter("%y-%m-%dT%H:%M")
    count = int(stat.loc["count"].values[0])
    mean = stat.loc["mean"].values[0]
    median = stat.loc["50%"].values[0]
    std = stat.loc["std"].values[0]
    textstr = "\n".join((f"count : {count}",
                     f"mean : {mean:.1f} mm",
                     f"median : {median:.1f} mm",
                     f"std : {std:.1f} mm",
                    ))


    

    N= len(df_night_pwv_curve)

    if N>10:
        # does the fit
        f = lambda x, *p: p[0] * x + p[1]
        x = df_night_pwv_curve["dt"].values
        y = df_night_pwv_curve["PWV [mm]_x"].values
        yerr = df_night_pwv_curve["PWV [mm]_err_x"].values
        n = len(y)
        
        #popt, pcov = optimize.curve_fit(f, x, y, [1,-4])
        fit_res = least_squares(funclineres,[5.,0],args = (x,y,yerr))
        popt = fit_res.x 
        npar = len(popt)
        J = fit_res.jac
        cov = np.linalg.inv(J.T.dot(J))
        chi2dof = ((funclineres(popt,x,y,yerr))**2).sum()/(n-npar)
        cov *= chi2dof
        perr = np.sqrt(np.diagonal(cov)) 
        

        xfit = np.linspace(x.min()*0.99,x.max()*1.05)
        yfit = funcline(popt,xfit)
        
        
       

        slope = popt[1]
        slope_err = perr[1]
        
        textstr2 = "\n".join((f"d(PWV/dt) : ",
                     f"slope : {slope:.3f} mm/hour ",
                     f"slope err : {slope_err:.3f} mm/hour",
                     f"chi2/ndeg : {chi2dof:.2f}",       
                    ))

        textstr3 = "\n".join((f"d(PWV/dt) : ",
                     f"slope : {slope:.3f} mm/hour ",
                     f"slope err : {slope_err:.3f} mm/hour",  
                    ))
    

        stat.loc["slope","PWV [mm]_x"] = slope 
        stat.loc["slope_err","PWV [mm]_x"] = slope_err 
        stat.loc["chi2","PWV [mm]_x"] = chi2dof
        
        # plot
        #fig,(ax,ax2) = plt.subplots(1,2,figsize=(12,5))
        
        fig = plt.figure(figsize=(14,5))
        #gs = GridSpec(1, 2, wspace=0, hspace=2., width_ratios=[2,1],figure=fig)
        gs = GridSpec(1, 2,  width_ratios=[2,1],figure=fig)

        ax = fig.add_subplot(gs[0])
        ax2 = fig.add_subplot(gs[1],sharey=ax1)
        #ax2 = fig.add_subplot(gs[1])
        
        df_night_pwv_curve.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,legend=False)
        ax.errorbar(x=df_night_pwv_curve.Time, y=df_night_pwv_curve["PWV [mm]_x"],yerr=df_night_pwv_curve["PWV [mm]_err_x"],fmt=".",color="r",ecolor="k")
        ax.set_ylabel("PWV [mm]_x")
        ax.set_xlabel("time")
        ax.xaxis.set_major_formatter(date_form)
        ax.set_ylim(0.,12.)
        ax.text(0.03, 0.95, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
        ax.text(0.45, 0.95, textstr3, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
        ax.set_title(f"night {night}")

        ax2.errorbar(x=df_night_pwv_curve.dt, y=df_night_pwv_curve["PWV [mm]_x"],yerr=df_night_pwv_curve["PWV [mm]_err_x"],fmt=".",color="r",ecolor="k")
        ax2.set_xlabel("hours")
        ax2.set_ylim(0.,12.)
        ax2.grid()
        ax2.plot(xfit,yfit,"k-")
        ax2.text(0.45, 0.95, textstr2, transform=ax2.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
        ax2.set_title(f"night {night}")
        
        plt.tight_layout()
        figname =f"{pathfigs}/pwv_per_night_{night}.pdf"
        plt.savefig(figname)
        plt.show()

        # add statistics
        all_dateObs_sel[night] = stat

## Make a summary of Night quality

In [ ]:
df = pd.DataFrame(columns = ["count","mean","median","std","slope","slope_err"])

In [ ]:
for nightObs, stat in all_dateObs_sel.items():
    count = int(stat.loc["count"].values[0])
    mean = stat.loc["mean"].values[0]
    median = stat.loc["50%"].values[0]
    std = stat.loc["std"].values[0]
    slope = stat.loc["slope"].values[0]
    slope_err = stat.loc["slope_err"].values[0]
    df.loc[nightObs] = [count,mean,median,std,slope,slope_err]
    

In [ ]:
def ComputeDataTimeStr(num):
    year = num//10000
    remain = (num-year*10000)
    month = remain//100
    day = remain-100*month
    yearstr = f"{year}"
    monthstr = f"{month}"
    daystr= f"{day}"
    stry = yearstr.zfill(4)
    strm = monthstr.zfill(2)
    strd = daystr.zfill(2)
    
    str = f"{stry}-{strm}-{strd}"
    return str

In [ ]:
df["Time"] = pd.to_datetime([ ComputeDataTimeStr(num) for  num in df.index])

In [ ]:
df.head()

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(2,1,figsize=(12,6),sharex=True)
ax1,ax2  = axs
df.plot(x="Time",y="median",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="PWV_x")
ax1.errorbar(x=df["Time"], y=df["median"],yerr=df["std"],fmt=".",color="r",ecolor="k")
ax1.set_ylabel("PWV (mm)")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("Median Precipitable water vapor per night")
ax1.set_ylim(0.,11)

ax2.set_title("Fitted drift per night")
ax2.errorbar(x=df["Time"], y=df["slope"],yerr=df["slope_err"],fmt=".",color="r",ecolor="k")
ax2.grid()
ax2.set_ylabel("dPWV/dt (mm/hour)")
ax2.set_xlabel("time")
ax2.set_ylim(-1.,1.)
ax2.xaxis.set_major_formatter(date_form)

ax2.tick_params(axis='x', labelrotation=45)
#for key, tt in d.items():
#    ax1.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(12,4))
ax1 = axs
df.plot(x="Time",y="median",ax=ax1,marker='+',c="r",ms=20,lw=0.0,grid=True,label="PWV_x")
ax1.errorbar(x=df["Time"], y=df["median"],yerr=df["std"],fmt="o",ms=5,color="r",ecolor="k")
ax1.set_ylabel("PWV (mm)")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("Median and spread of Precipitable water vapor per night")
ax1.set_ylim(0.,11)